In [ ]:
import tensorflow as tf
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold , StratifiedKFold

In [ ]:
df = pd.read_csv('../input/rfcx-species-audio-detection/train_tp.csv')

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def serialize_example(wav, recording_id, target, song_id, tmin,fmin, tmax, fmax):
  feature = {
      'wav': _bytes_feature(wav),
      'recording_id': _bytes_feature(recording_id),
      'target': _float_feature(target),
      'song_id': _float_feature(song_id),
      'tmin': _float_feature(tmin),
      'fmin' : _float_feature(fmin),
      'tmax': _float_feature(tmax),
      'fmax' : _float_feature(fmax),
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString() 

In [ ]:
tfrec_num = 0
kfold = GroupKFold(n_splits=10)
for fold, (train_idx, test_idx) in enumerate(kfold.split(df['recording_id'], df['species_id'], df['recording_id'])):
    x_train , y_train = df['recording_id'][test_idx] , df['species_id'][test_idx]
   
    with tf.io.TFRecordWriter('tp%.2i-%.2i.tfrec'%(tfrec_num, len(test_idx))) as writer:
        print('Writing_tfrecords ',fold)
        for recording_id , true_value in zip(x_train, y_train): 
            wav, _ = librosa.load(f'../input/rfcx-species-audio-detection/train/{recording_id}.flac', sr = None)
            label_info = df.loc[df['recording_id'] == str(recording_id)].values[0]
            wav = tf.audio.encode_wav(tf.reshape(wav,(wav.shape[0], 1)) ,sample_rate = 48000)
            recording_id = label_info[0].encode()
            target = label_info[1]
            song_id = label_info[2]
            tmin = label_info[3]
            fmin = label_info[4]
            tmax = label_info[5]
            fmax = label_info[6]
            example = serialize_example(wav, recording_id, target, song_id, tmin,fmin, tmax, fmax)
            writer.write(example)
    tfrec_num += 1

In [ ]:
kfold.get_n_splits(df['recording_id'], df['species_id'], df['recording_id'])